In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
import gensim.downloader

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/MyDrive/datasets/tweets.csv')

In [4]:
pd.set_option('display.max_colwidth',None)
data.head(10)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!
5,6,1,iPhone software update fucked up my phone big time Stupid iPhones
6,7,0,Happy for us .. #instapic #instadaily #us #sony #xperia #xperiaZ https://instagram.com/p/z9qGfWlvj7/
7,8,0,New Type C charger cable #UK http://www.ebay.co.uk/itm/-/112598674021 … #bay #Amazon #etsy New Year #Rob Cross #Toby Young #EVEMUN #McMafia #Taylor #SPECTRE 2018 #NewYear #Starting 2018 #recipes #technology #SamsungGalaxyS9 #iPhoneX pic.twitter.com/PjIwq59WtC
8,9,0,Bout to go shopping again listening to music #iphone #justme #music #likeforlike #followforfollow… http://instagr.am/p/Vj6bg5tLql/
9,10,0,Photo: #fun #selfie #pool #water #sony #camera #picoftheday #sun #instagood #boy #cute #outdoor... http://tmblr.co/ZAjC0n1ms_Gp8


In [5]:
def preprocess_tweet(tweet):

    tweet = tweet.lower()
    try:
        tweet = re.sub(r"(http|https):\/\/\S+\.\S+\/[^\s]+", "", tweet)
        tweet = re.sub(r"pic\.twitter\.com\/\w+[^\s]+", "", tweet)
        tweet = re.sub(r"@\w+[^\s]+", "", tweet)
    except:
      tweet = tweet

    return tweet

In [6]:
data['tweet_clean'] = data['tweet'].apply(preprocess_tweet)
data['tweet_clean'] = data['tweet_clean'].apply(lambda x:gensim.utils.simple_preprocess(x))

In [7]:
data.head(15)

,id,label,tweet,tweet_clean
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,"[fingerprint, pregnancy, test, android, apps, beautiful, cute, health, igers, iphoneonly, iphonesia, iphone]"
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,"[finally, transparant, silicon, case, thanks, to, my, uncle, yay, sony, xperia, sonyexperias]"
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,"[we, love, this, would, you, go, talk, makememories, unplug, relax, iphone, smartphone, wifi, connect]"
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,"[wired, know, george, was, made, that, way, iphone, cute, daventry, home]"
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,"[what, amazing, service, apple, won, even, talk, to, me, about, question, have, unless, pay, them, for, their, stupid, support]"
5,6,1,iPhone software update fucked up my phone big time Stupid iPhones,"[iphone, software, update, fucked, up, my, phone, big, time, stupid, iphones]"
6,7,0,Happy for us .. #instapic #instadaily #us #sony #xperia #xperiaZ https://instagram.com/p/z9qGfWlvj7/,"[happy, for, us, instapic, instadaily, us, sony, xperia, xperiaz]"
7,8,0,New Type C charger cable #UK http://www.ebay.co.uk/itm/-/112598674021 … #bay #Amazon #etsy New Year #Rob Cross #Toby Young #EVEMUN #McMafia #Taylor #SPECTRE 2018 #NewYear #Starting 2018 #recipes #technology #SamsungGalaxyS9 #iPhoneX pic.twitter.com/PjIwq59WtC,"[new, type, charger, cable, uk, bay, amazon, etsy, new, year, rob, cross, toby, young, evemun, mcmafia, taylor, spectre, newyear, starting, recipes, technology, samsunggalaxys, iphonex]"
8,9,0,Bout to go shopping again listening to music #iphone #justme #music #likeforlike #followforfollow… http://instagr.am/p/Vj6bg5tLql/,"[bout, to, go, shopping, again, listening, to, music, iphone, justme, music, likeforlike, followforfollow]"
9,10,0,Photo: #fun #selfie #pool #water #sony #camera #picoftheday #sun #instagood #boy #cute #outdoor... http://tmblr.co/ZAjC0n1ms_Gp8,"[photo, fun, selfie, pool, water, sony, camera, picoftheday, sun, instagood, boy, cute, outdoor]"


In [8]:
data['label'].unique()

array([0, 1])

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data['tweet_clean'], data['label'], test_size =0.3)

In [10]:
from gensim.models import Word2Vec
w2v_model = gensim.models.Word2Vec(x_train,min_count = 1)

In [11]:
words = w2v_model.wv.index_to_key

In [12]:
x_train_vec = []
x_test_vec = []

for ls in x_train:
  vec = []
  for word in ls:
    if word in words:
      vec.append(w2v_model.wv['word'])
  x_train_vec.append(np.array(vec))

In [13]:
for ls in x_test:
  vec = []
  for word in ls:
    if word in words:
      vec.append(w2v_model.wv['word'])
  x_test_vec.append(np.array(vec))

In [14]:
x_train_vec_avg = []
x_test_vec_avg = []

def cal_avg_vec(v):
  if v.size:
    return v.mean(axis = 0)
  else:
    np.zeros(100,dtype=float)

for v in x_train_vec:
  x_train_vec_avg.append(cal_avg_vec(v))

for v in x_test_vec:
  x_test_vec_avg.append(cal_avg_vec(v))

In [15]:
x_train_vec_avg = np.array(x_train_vec_avg)
x_test_vec_avg = np.array(x_test_vec_avg)

In [17]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(x_train_vec_avg,y_train)

SVC(kernel='linear')

In [18]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x_test_vec_avg)

In [19]:
accuracy_score(y_test,y_pred)

0.7403198653198653

In [27]:
review = "I just made another crazy purchase lol my theory is 'Work hard, Play Hard' Lol #iPad #Apple #Shopping http://instagr.am/p/NUyFHvyE7I/"

In [28]:
review = preprocess_tweet(review)

In [29]:
review = gensim.utils.simple_preprocess(review)

In [30]:
vec_review = np.array([w2v_model.wv[i] for i in review if i in words]).mean(axis = 0)

In [31]:
y_pred = clf.predict([vec_review])

In [32]:
y_pred

array([0])

**Glove Twitter **

In [33]:
glove_vect = gensim.downloader.load('glove-twitter-100')

In [34]:
words = glove_vect.index_to_key

In [ ]:
words

In [39]:
x_train_vec = []
x_test_vec = []

for ls in x_train:
  vec = []
  for word in ls:
    if word in words:
      vec.append(glove_vect['word'])
  x_train_vec.append(np.array(vec))

In [40]:
for ls in x_test:
  vec = []
  for word in ls:
    if word in words:
      vec.append(glove_vect['word'])
  x_test_vec.append(np.array(vec))

In [41]:
x_train_vec_avg = []
x_test_vec_avg = []

def cal_avg_vec(v):
  if v.size:
    return v.mean(axis = 0)
  else:
    np.zeros(100,dtype=float)

for v in x_train_vec:
  x_train_vec_avg.append(cal_avg_vec(v))

for v in x_test_vec:
  x_test_vec_avg.append(cal_avg_vec(v))

In [42]:
x_train_vec_avg = np.array(x_train_vec_avg)
x_test_vec_avg = np.array(x_test_vec_avg)

In [44]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(x_train_vec_avg,y_train)

SVC(kernel='linear')

In [45]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x_test_vec_avg)

In [46]:
accuracy_score(y_test,y_pred)

0.7403198653198653

In [59]:
review = 'Bout to go shopping again listening to music #iphone #justme #music #likeforlike #followforfollow… http://instagr.am/p/Vj6bg5tLql/'

In [60]:
review = preprocess_tweet(review)

In [61]:
review = gensim.utils.simple_preprocess(review)

In [62]:
vec_review = np.array([glove_vect[i] for i in review if i in words]).mean(axis = 0)

In [63]:
y_pred = clf.predict([vec_review])

In [64]:
y_pred

array([0])